In [1]:
import gym
import pybullet_envs

import ray
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from ray.tune.registry import register_env
import time

from low_level_env import LowLevelHumanoidEnv

import timeit

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
ray.shutdown()
ray.init(ignore_reinit_error=True)

2021-03-23 22:11:20,723	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.108',
 'raylet_ip_address': '192.168.0.108',
 'redis_address': '192.168.0.108:6379',
 'object_store_address': '/tmp/ray/session_2021-03-23_22-11-20_326777_139500/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-03-23_22-11-20_326777_139500/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-03-23_22-11-20_326777_139500',
 'metrics_export_port': 65414,
 'node_id': '1675e74bb356778cabe855df1d689699ab645d6b'}

In [3]:
def make_env(env_config):
    import pybullet_envs
    return LowLevelHumanoidEnv()


ENV = 'HumanoidBulletEnv-v0-Low'
register_env(ENV, make_env)
TARGET_REWARD = 2000

In [4]:
# def make_env(env_config):
#     import pybullet_envs
#     return gym.make('HumanoidBulletEnv-v0')

# ENV = 'HumanoidBulletEnv-v0'
# register_env(ENV, make_env)
# TARGET_REWARD = 2000

In [5]:
# Train
# tune.run(
#     PPOTrainer,
#     name="HWalk",
#     resume=False,
#     checkpoint_at_end=True,
#     checkpoint_freq=5,
#     keep_checkpoints_num=50,
#     checkpoint_score_attr="episode_reward_max",
#     stop={"episode_reward_mean": TARGET_REWARD},
#     config={
#         "env": ENV,
#         "num_workers": 12,
#         "num_gpus": 1,
#         "monitor": True,
#         "evaluation_num_episodes": 50,
#         "gamma": 0.995,
#         "lambda": 0.95,
#         "clip_param": 0.2,
#         "kl_coeff": 1.0,
#         "num_sgd_iter": 20,
#         "lr": .0001,
#         "sgd_minibatch_size": 8000,
#         "train_batch_size": 24000,
#         "model": {
#             "fcnet_hiddens": [256, 128, 64],
#             "fcnet_activation": "tanh",
#             "free_log_std": True,
#         },
#         "batch_mode": "complete_episodes",
#         "observation_filter": "MeanStdFilter",
#     }
# )

In [6]:
config = {
    "env": ENV,
    "num_workers": 0,
    "num_gpus": 1,
    "monitor": False,
    "evaluation_num_episodes": 50,
    "gamma": 0.995,
    "lambda": 0.95,
    "clip_param": 0.2,
    "kl_coeff": 1.0,
    "num_sgd_iter": 20,
    "lr": .0001,
    "sgd_minibatch_size": 8000,
    "train_batch_size": 24000,
    "model": {
        "fcnet_hiddens": [256, 128, 64],
        "fcnet_activation": "tanh",
        "free_log_std": True,
    },
    "batch_mode": "complete_episodes",
    "observation_filter": "MeanStdFilter",
}

In [7]:
agent = PPOTrainer(config)

2021-03-23 22:11:25,720	INFO trainer.py:591 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-03-23 22:11:25,721	INFO trainer.py:616 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'default_policy/log_std:0' shape=(17,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
2021-03-23 22:11:28,828	WARNING util.py:43 -- Install gputil for GPU system monitoring.


In [8]:
# agent.restore("/home/aditya/ray_results/HWalk/PPO_HumanoidBulletEnv-v0_881b4_00000_0_2021-02-21_17-45-29/checkpoint_710/checkpoint-710")
# agent.restore("/home/aditya/ray_results/HWalk_Low/PPO_HumanoidBulletEnv-v0-Low_92895_00000_0_2021-03-20_09-42-47/checkpoint_430/checkpoint-430")
# agent.restore("/home/aditya/ray_results/HWalk_Low/PPO_HumanoidBulletEnv-v0-Low_5ab1c_00000_0_2021-03-20_11-42-54/checkpoint_805/checkpoint-805")
agent.restore("/home/aditya/ray_results/HWalk_Low_Mimic/PPO_HumanoidBulletEnv-v0-Low_6a5d5_00000_0_2021-03-23_20-29-30/checkpoint_535/checkpoint-535")

2021-03-23 22:11:48,764	INFO trainable.py:328 -- Restored on 192.168.0.108 from checkpoint: /home/aditya/ray_results/HWalk_Low_Mimic/PPO_HumanoidBulletEnv-v0-Low_6a5d5_00000_0_2021-03-23_20-29-30/checkpoint_535/checkpoint-535
2021-03-23 22:11:48,765	INFO trainable.py:336 -- Current state after restoring: {'_iteration': 535, '_timesteps_total': None, '_time_total': 5562.900058031082, '_episodes_total': 184568}


In [9]:
agent.get_weights()['default_policy']['default_policy/fc_1/kernel'].shape

(44, 256)

In [10]:
env.close()

NameError: name 'env' is not defined

In [11]:
env = make_env(ENV)

In [12]:
env.render()
observation = env.reset()

In [12]:
agent.compute_action(observation)

array([ 0.14176384,  1.        , -0.6668723 ,  1.        , -0.49632737,
       -1.        ,  0.7887472 ,  0.40421596,  0.10032001,  1.        ,
       -0.37429866, -0.2658472 ,  1.        ,  0.5843158 , -0.34374365,
        0.8275212 , -0.2380859 ], dtype=float32)

In [16]:
rew = []
for i in range(5):
    env.render()
    observation = env.reset()
    done = False
    tempRew = []
    startFrame = env.frame
    start = timeit.default_timer()
    while(not done):
        action = agent.compute_action(observation)
        observation, reward, done, info = env.step(action)
        # tempRew.append(env.calcJointScore())
        time.sleep(1.0/60)
    # stop = timeit.default_timer()
    # rew.append((startFrame, tempRew, stop-start))

In [17]:
env.close()

In [18]:
ray.shutdown()

In [43]:
rew[0][2]

23.287814890005393

In [44]:
len(rew[0][1])

1000

In [45]:
1000 / 23.28

42.955326460481096

In [47]:
1000 * 15/23

652.1739130434783

In [50]:
for r in rew:
    print(r[0], len(r[1]))

235 640
125 1000
210 41
140 56
117 1000
46 1000
227 78
47 1000
169 1000
187 43


In [56]:
start = timeit.default_timer()
time.sleep(3.2873985)
stop = timeit.default_timer()
print(stop-start)

3.2890066819963977
